In [1]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("nmplot.ipynb"))))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from TVA1_UU import TVA1_UU as BHDVCS
import plotly.graph_objects as go
from scipy.stats import norm

dats = pd.read_csv('../dvcs_xs_May-2021_342_sets_with_trueCFFs.csv')
n = np.array(dats['#Set'])
ind = np.array(dats['index'])
k = np.array(dats['k'])
qq = np.array(dats['QQ'])
xb = np.array(dats['x_b'])
t = np.array(dats['t'])
phi = np.array(dats['phi_x'])
F = np.array(dats['F'])
errF = np.array(dats['sigmaF'])
varF = np.array(dats['varF']) 
F1 = np.array(dats['F1'])
F2 = np.array(dats['F2'])
const = np.array(dats['dvcs'])
trueReH = np.array(dats['ReH'])
trueReE = np.array(dats['ReE'])
trueReHT = np.array(dats['ReHTilde'])

def getSet(sets, setNum, itemsInSet=45):
    a = setNum*itemsInSet
    b = (setNum+1)*itemsInSet-1
    return sets[a:b]

bhdvcs = BHDVCS()

In [2]:
def cffplot(setNum, repNums, results, col):
    types = {
        "ReH": 0,
        "ReE": 1,
        "ReHT": 2
    }
    fig = go.Figure()
    otherfig = go.Figure()
    for repNum in repNums:
        arr = results.loc[(setNum,repNum)]
        epochs = arr['epoch'].unique()
        fig.add_trace(go.Scatter(x=epochs, y=arr[col].values, mode='lines', name = str(repNum)))
    realReHT = getSet(trueReHT, setNum)[types[col]]
    fig.add_trace(go.Scatter(x=epochs, y=[realReHT] * epochs.size, mode='lines', name = 'Real'))
    fig.update_layout(template="plotly_white",title_text="Set "+str(setNum+1),showlegend=True,width=800,height=800)
    fig.show()
    
def final(setNum, repNums, results, col):
    guh = []
    for repNum in repNums:
        arr = results.loc[(setNum,repNum)]
        epoch = arr['epoch'].max()
        val = arr.iloc[-1][col]
        guh.append(val)
    return np.array(guh)

def readResults(sets, path):
    results = pd.read_csv(path+'ResultsSet'+str(sets)+'.csv')
    results = results.set_index(['set','replica'])
    return results

path = "combinedresults/"

def stats(sets, col):
    return norm.fit(final(sets,[x for x in range(500)], readResults(sets, path), col))

def plot(sets, col):
    cffplot(sets,[x for x in range(500)], readResults(sets,path), col)

In [38]:

df = pd.DataFrame([])
for x in range(100):
    #row = pd.DataFrame([])
    #row['set'] = x+1
    reh, reh_error = stats(x,"ReH")
    ree, ree_error = stats(x,"ReE")
    reht, reht_error = stats(x,"ReHT")
    row = pd.DataFrame(np.reshape([x+1, reh, reh_error, ree, ree_error, reht, reht_error],(1,7)), columns = ['set', 'ReH', 'ReH_error', 'ReE', 'ReE_error', 'ReHtilde', 'ReHtilde_error'])
    df = pd.concat([df, row])
df.head()

,set,ReH,ReH_error,ReE,ReE_error,ReHtilde,ReHtilde_error
0,1.0,1.265474,1.392240,-36.149775,88.884900,4.515400,1.737736
0,2.0,1.413367,1.840714,-2.289463,50.950474,6.037459,10.718163
0,3.0,4.041066,6.354663,-22.181061,57.538671,18.714607,35.535863
0,4.0,-3.438236,6.525619,54.103238,70.729897,-11.721351,21.236606
0,5.0,19.390245,312.542774,-248.657201,4477.014462,141.221059,2457.236945


In [39]:
df.to_csv('nmresultscombinedwitherror.csv')

In [ ]:
cffplot(1,[x for x in range(60)],results2, "ReE")
cffplot(1,[x for x in range(60)],sciresults2, "ReE")